<a href="https://colab.research.google.com/github/yogapande4/python-colab-project/blob/main/UTS_Komputasi_Paralel_dan_Terdistribusi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install psutil

In [ ]:

from re import T
import os, time, psutil
from threading import Thread, Event
from multiprocessing import Process, cpu_count

def cpu_bound(n=20_000_000):
  i=0
  for _ in range(n):
    i += 1
  return i

def start_cpu_monitor():
  stop = Event()
  def _watch():
    psutil.cpu_percent(interval=None, percpu=True)
    while not stop.is_set():
      loads = psutil.cpu_percent(interval=0.2, percpu=True)
      bars = " | ".join(f"core {i}: {int(v):3d}%" for i, v in enumerate(loads))
      print(bars)
  t = Thread(target=_watch, daemon=True)
  t.start()
  return stop, t

def time_it(fn):
  t0 = time.perf_counter()
  fn()
  return time.perf_counter() - t0

import threading

print(f"Logical CPUs seen by Colab: {psutil.cpu_count(logical=True)}")

def run_threads():
  threads = [threading.Thread(target=cpu_bound) for _ in range(5)]
  for t in threads: t.start()
  for t in threads: t.join()

stop, mon = start_cpu_monitor()
dt_threads = time_it(run_threads)
stop.set(); mon.join()
print(f"\nThreads time (5 threads): {dt_threads:.2f}s")

Logical CPUs seen by Colab: 2
core 0:  60% | core 1:  51%
core 0:  56% | core 1:  44%
core 0:  49% | core 1:  57%
core 0:  54% | core 1:  59%
core 0:  59% | core 1:  44%
core 0:  56% | core 1:  60%
core 0:  54% | core 1:  51%
core 0:  50% | core 1:  59%
core 0:  50% | core 1:  37%

Threads time (5 threads): 5.01s


In [ ]:
#Process (Spawn)
import multiprocessing as mp
import os, time

def worker(name):
  print(f"[spawn] Child PID: {os.getpid()} | Name: {name}", flush=True)
  time.sleep(1)

def run_with_spawn():
  ctx = mp.get_context("spawn")
  print("\n=== Using 'spawn' method ===")
  processes = []
  for i in range(3):
    p = ctx.Process(target=worker, args=(f"Process-{i}",))
    p.start()
    processes.append(p)
  for p in processes:
    p.join()
  print("All processes finished with 'spawn'")

if __name__ == "__main__":
  run_with_spawn()


=== Using 'spawn' method ===
All processes finished with 'spawn'


In [ ]:
#Process (fork)
import multiprocessing as mp
import os, time

def worker(name):
  print(f"[fork] Child PID: {os.getpid()} | Name: {name}", flush=True)
  time.sleep(1)

def run_with_fork():
  ctx = mp.get_context("fork")
  print(f"\n=== Using 'fork' method ===")
  processes =[]
  for i in range(3):
    p = ctx.Process(target=worker, args=(f"Process-{i}",))
    p.start()
    processes.append(p)
  for p in processes:
    p.join()
  print("All processes finished with 'fork'")

if __name__ == "__main__":
  run_with_fork()


=== Using 'fork' method ===
[fork] Child PID: 4896 | Name: Process-0
[fork] Child PID: 4899 | Name: Process-1
[fork] Child PID: 4904 | Name: Process-2
All processes finished with 'fork'


In [ ]:
#Process (forkserver)
import multiprocessing as mp
import os, time

def worker(name):
  print(f"[forkserver] Child PID: {os.getpid()} | Name: {name}", flush=True)
  time.sleep(1)

def run_with_forkserver():
  ctx = mp.get_context("forkserver")
  print(f"\n=== Using 'forkserver' method ===")
  processes = []
  for i in range(3):
    p = ctx.Process(target=worker, args=(f"Process-{i}",))
    p.start()
    processes.append(p)
  for p in processes:
    p.join()
  print("All processes finished with 'forkserver'")

if __name__ == "__main__":
  run_with_forkserver()


=== Using 'forkserver' method ===
All processes finished with 'forkserver'


In [ ]:
#process (fork, spawn, forkserver)
import multiprocessing as mp

print("Default start method:", mp.get_start_method())
print("Available methods:", mp.get_all_start_methods())

Default start method: fork
Available methods: ['fork', 'spawn', 'forkserver']


In [ ]:
import multiprocessing
import os, time

def worker(name):
  print(f"Child PID: {os.getpid()} | Name: {name}", flush=True)
  time.sleep(1)

def run_with(method):
  ctx = mp.get_context(method)
  print(f"\n=== Using '{method}' method ===")
  processes = []
  for i in range(3):
    p = ctx.Process(target=worker, args=(f"Process-{i}",))
    p.start()
    processes.append(p)
  for p in processes:
    p.join()
  print(f"All processes finished with '{method}' \n")

if __name__ == "__main__":
  for method in mp.get_all_start_methods():
    run_with(method)


=== Using 'fork' method ===
Child PID: 7251 | Name: Process-0
Child PID: 7254 | Name: Process-1
Child PID: 7263 | Name: Process-2
All processes finished with 'fork' 


=== Using 'spawn' method ===
All processes finished with 'spawn' 


=== Using 'forkserver' method ===
All processes finished with 'forkserver' 



In [ ]:
import multiprocessing as mp

x = 10

def modify_value():
  global x
  x += 5
  print("Child x:", x, flush=True)

if __name__ == "__main__":
  p = mp.Process(target=modify_value)
  p.start()
  p.join()
  print("Parent x:", x, flush=True)

  for method in mp.get_all_start_methods():
    ctx = mp.get_context(method)
    print(f"\n--- {method.upper()} ---")
    p = ctx.Process(target=modify_value)
    p.start()
    p.join()
    print("Parent x:",)

Child x: 15
Parent x: 10

--- FORK ---
Child x: 15
Parent x: 10

--- SPAWN ---
Parent x: 10

--- FORKSERVER ---
Parent x: 10


In [ ]:
#process (starttime)
import time, os

def worker(n=2_000_000):
  s = 0
  for _ in range(n):
    s += 1

  print(f"PID={os.getpid()} done", flush=True)


In [ ]:
import multiprocessing as mp
import time

def bench_method(method: str, nproc=4, n=2_000_000):
  ctx = mp.get_context(method)
  def run_once():
    t0 = time.perf_counter()
    procs = [ctx.Process(target=worker, args=(n,)) for _ in range(nproc)]
    for p in procs: p.start()
    for p in procs: p.join()
    return time.perf_counter() - t0

  print(f"\n=== {method.upper()} (nproc={nproc}) ===")
  cold = run_once()
  warm = run_once()
  print(f"Cold time : {cold:.3f}s")
  print(f"Warm time : {warm:.3f}s")
  return cold, warm

if __name__ == "__main__":
  print("Default start method:", mp.get_start_method())
  print("Available methods:", mp.get_all_start_methods())

  results = {}
  for m in mp.get_all_start_methods():
    try:
      results[m] = bench_method(m, nproc=min(4, mp.cpu_count()))
    except (RuntimeError, AttributeError) as e:
      print(f"Skipping {m}: {e}")

  print("\nSummary (seconds): ")
  for m, (cold, warm) in results.items():
    print(f"- {m:10s} cold={cold:6.3f} warm={warm:6.3f}")


Default start method: fork
Available methods: ['fork', 'spawn', 'forkserver']

=== FORK (nproc=2) ===
PID=13653 done
PID=13654 done
PID=13663 done
PID=13664 done
Cold time : 0.245s
Warm time : 0.246s

=== SPAWN (nproc=2) ===
Cold time : 0.131s
Warm time : 0.179s

=== FORKSERVER (nproc=2) ===
Cold time : 0.016s
Warm time : 0.010s

Summary (seconds): 
- fork       cold= 0.245 warm= 0.246
- spawn      cold= 0.131 warm= 0.179
- forkserver cold= 0.016 warm= 0.010


In [ ]:
#memory sharing (not thread)
import json
import urllib.request
import time

def count_letters (url, frequency):
  response = urllib.request.urlopen(url)
  txt = str(response.read())
  for l in txt:
    letter = l.lower ()
    if letter in frequency:
      frequency[letter] += 1

def main():
  frequency = {}
  for c in "abcdefghijklmnopqrstuvwxyz":
    frequency[c] = 0
  start=time.time()
  for i in range(9800, 9820):
    count_letters(f"https://www.rfc-editor.org/rfc/rfc{i}.txt", frequency)
  end = time.time()
  print(json.dumps (frequency, indent=4))
  print("Done, time taken", end-start)

main()

{
    "a": 56741,
    "b": 14195,
    "c": 39527,
    "d": 34707,
    "e": 103451,
    "f": 24647,
    "g": 15270,
    "h": 26847,
    "i": 64442,
    "j": 1305,
    "k": 7349,
    "l": 27361,
    "m": 18921,
    "n": 85702,
    "o": 52261,
    "p": 24789,
    "q": 2341,
    "r": 53278,
    "s": 57659,
    "t": 74010,
    "u": 20797,
    "v": 8793,
    "w": 8430,
    "x": 4023,
    "y": 11091,
    "z": 1260
}
Done, time taken 1.1932404041290283


In [ ]:
#memory sharing (Thread)
import json
import urllib.request
import time
from threading import Thread, Lock

finished_count = 0

def count_letters (url, frequency, mutex):
  response = urllib.request.urlopen(url)
  txt = str(response.read())
  mutex.acquire()
  for l in txt:
    letter = l.lower()
    if letter in frequency:
      frequency[letter] += 1
  global finished_count
  finished_count += 1
  mutex.release()

def main():
  frequency = {}
  mutex = Lock()
  for c in "abcdefghijklmnopqrstuvwxyz":
    frequency[c] = 0
  start = time.time()
  for i in range(9800, 9820):
    t = Thread(target=count_letters, args=(f"https://www.rfc-editor.org/rfc/rfc{i}.txt", frequency, mutex)).start()
  while True:
    mutex.acquire()
    if finished_count == 20:
      break
    mutex.release()
    time.sleep(0.5)
  end = time.time()
  print(json.dumps(frequency, indent=4))
  print("Done, time taken", end - start)

main()

{
    "a": 56741,
    "b": 14195,
    "c": 39527,
    "d": 34707,
    "e": 103451,
    "f": 24647,
    "g": 15270,
    "h": 26847,
    "i": 64442,
    "j": 1305,
    "k": 7349,
    "l": 27361,
    "m": 18921,
    "n": 85702,
    "o": 52261,
    "p": 24789,
    "q": 2341,
    "r": 53278,
    "s": 57659,
    "t": 74010,
    "u": 20797,
    "v": 8793,
    "w": 8430,
    "x": 4023,
    "y": 11091,
    "z": 1260
}
Done, time taken 0.6115095615386963


In [ ]:
#Thread Synchronize (tanpa mutex)
import time
from threading import Thread

class PelitBoros:
  money = 100

  def pelit(self):
    for i in range(100000000):
      self.money += 10
    print("Boros Selesai")

  def boros(self):
    for i in range(100000000):
      self.money -= 10
    print("Boros Selesai")

ss = PelitBoros()
Thread(target=ss.pelit).start()
Thread(target=ss.boros).start()
time.sleep(10)
print("Money in the end", ss.money)

Money in the end -3393670


In [ ]:
#Thread Synchronize (dengan mutex)

import time
from threading import Thread, Lock, Event

class PelitBoros:
  def __init__(self, loops=100000000):
    self.money = 100
    self.mutex = Lock()
    self.loops = loops
    self.pelit_selesai = Event()
    self.boros_selesai = Event()

  def pelit(self):
    for _ in range(self.loops):
      with self.mutex:
        self.money += 10
    print("Pelit Done")
    self.pelit_selesai.set()

  def boros(self):
    for _ in range(self.loops):
      with self.mutex:
        self.money -= 10
    print("Boros Done")
    self.boros_selesai.set()

ss = PelitBoros()
Thread(target=ss.pelit).start()
Thread(target=ss.boros).start()

ss.pelit_selesai.wait()
ss.boros_selesai.wait()

print("Money in the end", ss.money)

Boros Done
Pelit Done
Money in the end 100
